<a href="https://colab.research.google.com/github/TegaEth/internship-AI/blob/main/internship_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain langgraph openai unstructured pypdf python-pptx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 24.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 80.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 99.0 MB/s eta 0:00:00
  

In [4]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.44
    Uninstalling langsmith-0.3.44:
      Successfully uninstalled langsmith-0.3.44
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.63
    Uninstalling langchain-core-0.3.63:
      Successfully uninstalled langchain-core-0.3.63


In [5]:
!pip install langchain_cerebras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 2.8 MB/s eta 0:00:00


In [7]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=602aacbe46d7e06a232e267550bafcf073808df4afe3b07ecf428dc47e9cce34
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [25]:
import os
print(os.listdir())  # Should show your uploaded file

['.config', '4.pdf', '5.pdf', '.git', '.ipynb_checkpoints', '6.pdf', 'sample_data']


In [83]:
from langchain.document_loaders import PyPDFLoader, UnstructuredPowerPointLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langgraph.graph import StateGraph, END
from langchain_core.runnables import RunnableLambda, RunnableMap
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_cerebras import ChatCerebras
from fpdf import FPDF
import os
import random
from datetime import datetime

In [84]:
#llmm to be used
llm = ChatCerebras(
    model="llama-3.3-70b",
    api_key="csk-ph49yn4383r2rrhh6ch4kjrhddfjd3frt98npmecmd5dknf5"
)

In [112]:
# Node 1: Upload Handler (Multiple PDF or PPTX files)

def load_documents(file_paths: list[str]) -> str:
    all_text = []

    for file_path in file_paths:
        try:
            if file_path.endswith(".pdf"):
                loader = PyPDFLoader(file_path)
            elif file_path.endswith(".pptx"):
                loader = UnstructuredPowerPointLoader(file_path)
            else:
                print(f"Warning: Unsupported file format for {file_path}")
                continue

            pages = loader.load()
            file_text = "\n".join([page.page_content for page in pages])
            all_text.append(f"--- Content from {os.path.basename(file_path)} ---\n{file_text}")

        except Exception as e:
            print(f"Error loading {file_path}: {str(e)}")
            continue

    return "\n\n".join(all_text)

In [123]:
# add summary
def inject_summary(material: str, lecturer_summary: str = None) -> str:
    # Don't inject the summary into the material itself
    # The summary will be used in the question generation prompt instead
    return material

In [124]:
# chunking and summarization
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
)

def chunk_content(text: str) -> list[str]:
    return text_splitter.split_text(text)

In [125]:
# Node 4: Question Generator Node

question_prompt_template = PromptTemplate(
    input_variables=["chunk"],
    template="""
You are an exam generator AI. Based on the following content, generate 2 multiple-choice questions:

---
{chunk}
---

Generate questions that:
- Test understanding of specific concepts, definitions, formulas, and processes
- Ask directly about the subject matter without referencing "the material", "the text", or "the lecture"
- Include 1 correct answer and 3 plausible, challenging distractors
- Are clearly worded and academically appropriate

AVOID phrases like:
- "according to the material"
- "in the given text"
- "the lecture states"
- "based on the material"
- "the document mentions"

Instead, ask direct questions about the concepts themselves.

Format each question EXACTLY like this:
Q: [Direct question about the concept]
A) Option A
B) Option B
C) Option C
D) Option D
Correct Answer: [A/B/C/D]
Explanation: [Short explanation of why this answer is correct]

---

Q: [Second direct question about the concept]
A) Option A
B) Option B
C) Option C
D) Option D
Correct Answer: [A/B/C/D]
Explanation: [Short explanation of why this answer is correct]
"""
)

def generate_questions(chunk: str, summary: str = None) -> str:
    # Add summary guidance to the prompt without including it in the content
    summary_instruction = ""
    if summary:
        summary_instruction = f"\nIMPORTANT: {summary}\n"

    prompt = f"""
You are an exam generator AI. Based on the following content, generate 2 multiple-choice questions:

---
{chunk}
---
{summary_instruction}
Generate questions that:
- Test understanding of specific concepts, definitions, formulas, and processes
- Ask directly about the subject matter without referencing "the material", "the text", or "the lecture"
- Include 1 correct answer and 3 plausible, challenging distractors
- Are clearly worded and academically appropriate

AVOID phrases like:
- "according to the material"
- "in the given text"
- "the lecture states"
- "based on the material"
- "the document mentions"

Instead, ask direct questions about the concepts themselves.

Format each question EXACTLY like this:
Q: [Direct question about the concept]
A) Option A
B) Option B
C) Option C
D) Option D
Correct Answer: [A/B/C/D]
Explanation: [Short explanation of why this answer is correct]

---

Q: [Second direct question about the concept]
A) Option A
B) Option B
C) Option C
D) Option D
Correct Answer: [A/B/C/D]
Explanation: [Short explanation of why this answer is correct]
"""

    result = llm.invoke(prompt)
    # Fix: Extract content from AIMessage object
    if hasattr(result, 'content'):
        return result.content
    else:
        return str(result)

In [126]:
# Retry Logic Node

def retry_generation(state):
    if not state["raw_outputs"]:
        raise ValueError("Empty output. Retry generation.")
    return state


In [127]:
# Subject Classification Node

def classify_subject(material: str) -> str:
    prompt = f"""
You are a classifier AI. Based on the following content, identify the academic subject (e.g., Biology, Economics, Physics, Computer Science, Project Management, etc.):

{material[:2000]}...

Respond with just the subject name.
"""
    result = llm.invoke(prompt)
    # Fix: Extract content from AIMessage object
    if hasattr(result, 'content'):
        return result.content.strip()
    else:
        return str(result).strip()

In [128]:
# Enhanced Output Formatter Node

def format_output(raw_outputs: list[str]) -> dict:
    questions = []
    for output in raw_outputs:
        # Split by "Q:" and process each question
        parts = output.split("Q:")
        for part in parts[1:]:  # Skip first empty part
            if part.strip():
                questions.append(f"Q:{part.strip()}")

    # Limit to 20 questions and ensure they're properly formatted
    final_questions = []
    for q in questions[:20]:
        if "Correct Answer:" in q and "Explanation:" in q:
            final_questions.append(q)

    return {"questions": final_questions}

In [129]:
# Enhanced PDF Export Nodes

def create_exam_header(pdf, course_info):
    """Create a professional header for the exam"""
    pdf.set_font("Arial", "B", 16)
    pdf.cell(0, 10, f"{course_info['course_code']}: {course_info['course_name']}", ln=True, align='C')
    pdf.ln(5)

    pdf.set_font("Arial", "B", 14)
    pdf.cell(0, 10, "EXAMINATION", ln=True, align='C')
    pdf.ln(5)

    pdf.set_font("Arial", "", 12)
    pdf.cell(0, 8, f"Instructor: {course_info['lecturer_name']}", ln=True)
    pdf.cell(0, 8, f"Date: {datetime.now().strftime('%B %d, %Y')}", ln=True)
    pdf.ln(10)

    pdf.set_font("Arial", "B", 12)
    pdf.cell(0, 8, "Instructions: Choose the best answer for each question.", ln=True)
    pdf.ln(10)

def export_questions_only_pdf(questions: list[str], course_info: dict, filename="exam_questions.pdf") -> str:
    """Export PDF with questions only (no answers or explanations)"""
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()

    # Add header
    create_exam_header(pdf, course_info)

    pdf.set_font("Arial", "", 12)

    question_num = 1
    for q in questions:
        lines = q.split('\n')

        # Process each question
        for line in lines:
            if line.startswith('Q:'):
                pdf.set_font("Arial", "B", 12)
                question_text = line[2:].strip()
                try:
                    pdf.multi_cell(0, 8, f"{question_num}. {question_text}", 0, 'L')
                except:
                    pdf.multi_cell(0, 8, f"{question_num}. {question_text.encode('ascii', 'replace').decode('ascii')}", 0, 'L')
                pdf.ln(3)
                pdf.set_font("Arial", "", 12)
                question_num += 1

            elif line.strip().startswith(('A)', 'B)', 'C)', 'D)')):
                try:
                    pdf.multi_cell(0, 6, f"   {line.strip()}", 0, 'L')
                except:
                    pdf.multi_cell(0, 6, f"   {line.strip().encode('ascii', 'replace').decode('ascii')}", 0, 'L')

            elif line.startswith('Correct Answer:') or line.startswith('Explanation:'):
                break  # Stop processing when we hit answers/explanations

        pdf.ln(5)

    output_path = os.path.join("./", filename)
    pdf.output(output_path)
    return output_path

def export_full_pdf(questions: list[str], course_info: dict, filename="exam_with_answers.pdf") -> str:
    """Export PDF with questions, answers, and explanations"""
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()

    # Add header
    create_exam_header(pdf, course_info)

    pdf.set_font("Arial", "", 12)

    question_num = 1
    for q in questions:
        lines = q.split('\n')

        # Process each question
        for line in lines:
            if line.startswith('Q:'):
                pdf.set_font("Arial", "B", 12)
                question_text = line[2:].strip()
                try:
                    pdf.multi_cell(0, 8, f"{question_num}. {question_text}", 0, 'L')
                except:
                    pdf.multi_cell(0, 8, f"{question_num}. {question_text.encode('ascii', 'replace').decode('ascii')}", 0, 'L')
                pdf.ln(3)
                pdf.set_font("Arial", "", 12)
                question_num += 1

            elif line.strip().startswith(('A)', 'B)', 'C)', 'D)')):
                try:
                    pdf.multi_cell(0, 6, f"   {line.strip()}", 0, 'L')
                except:
                    pdf.multi_cell(0, 6, f"   {line.strip().encode('ascii', 'replace').decode('ascii')}", 0, 'L')

            elif line.startswith('Correct Answer:'):
                pdf.set_font("Arial", "B", 11)
                try:
                    pdf.multi_cell(0, 6, f"   {line.strip()}", 0, 'L')
                except:
                    pdf.multi_cell(0, 6, f"   {line.strip().encode('ascii', 'replace').decode('ascii')}", 0, 'L')
                pdf.set_font("Arial", "", 12)

            elif line.startswith('Explanation:'):
                pdf.set_font("Arial", "I", 11)
                try:
                    pdf.multi_cell(0, 6, f"   {line.strip()}", 0, 'L')
                except:
                    pdf.multi_cell(0, 6, f"   {line.strip().encode('ascii', 'replace').decode('ascii')}", 0, 'L')
                pdf.set_font("Arial", "", 12)

        pdf.ln(8)

    output_path = os.path.join("./", filename)
    pdf.output(output_path)
    return output_path

In [132]:
# Build Enhanced LangGraph

from typing import TypedDict, List

class ExamState(TypedDict):
    file_paths: List[str]
    lecturer_name: str
    course_code: str
    course_name: str
    summary: str
    text: str
    chunks: List[str]
    raw_outputs: List[str]
    questions: List[str]
    questions_only_pdf: str
    full_pdf: str
    subject: str

def build_exam_graph():
    workflow = StateGraph(ExamState)

    workflow.add_node("load_files", RunnableLambda(lambda state: {"text": load_documents(state["file_paths"])}))

    workflow.add_node("add_summary", RunnableLambda(lambda state: {"text": inject_summary(state["text"], state.get("summary"))}))

    workflow.add_node("classify", RunnableLambda(lambda state: {"subject": classify_subject(state["text"])}))

    workflow.add_node("chunk", RunnableLambda(lambda state: {"chunks": chunk_content(state["text"])}))

    workflow.add_node("generate", RunnableLambda(lambda state: {
        "raw_outputs": [generate_questions(chunk, state.get("summary")) for chunk in state["chunks"][:10]]  # Pass summary to generation
    }))

    workflow.add_node("retry", RunnableLambda(retry_generation))

    workflow.add_node("format", RunnableLambda(lambda state: format_output(state["raw_outputs"])))

    workflow.add_node("export_pdfs", RunnableLambda(lambda state: {
        "course_info": {
            "lecturer_name": state["lecturer_name"],
            "course_code": state["course_code"],
            "course_name": state["course_name"]
        },
        "questions_only_pdf": export_questions_only_pdf(
            state["questions"],
            {
                "lecturer_name": state["lecturer_name"],
                "course_code": state["course_code"],
                "course_name": state["course_name"]
            },
            f"{state['course_code']}_exam_questions.pdf"
        ),
        "full_pdf": export_full_pdf(
            state["questions"],
            {
                "lecturer_name": state["lecturer_name"],
                "course_code": state["course_code"],
                "course_name": state["course_name"]
            },
            f"{state['course_code']}_exam_with_answers.pdf"
        ),
        "questions": state["questions"],
        "subject": state["subject"]
    }))

    # Define edges
    workflow.set_entry_point("load_files")
    workflow.add_edge("load_files", "add_summary")
    workflow.add_edge("add_summary", "classify")
    workflow.add_edge("classify", "chunk")
    workflow.add_edge("chunk", "generate")
    workflow.add_edge("generate", "retry")
    workflow.add_edge("retry", "format")
    workflow.add_edge("format", "export_pdfs")
    workflow.add_edge("export_pdfs", END)

    return workflow.compile()

In [133]:
# Example Run

if __name__ == "__main__":
    graph = build_exam_graph()

    input_data = {
        "file_paths": ["5.pdf", "6.pdf","4.pdf"],  # List of multiple files
        "lecturer_name": "Dr. John Bush Idoko",
        "course_code": "ECC427",
        "course_name": "Business for Engineers",
        "summary": "Focus on questions that test student understanding and not ability to remembr facts"
    }

    result = graph.invoke(input_data)
    print(f"\nCourse: {input_data['course_code']} - {input_data['course_name']}")
    print(f"Instructor: {input_data['lecturer_name']}")
    print(f"Subject: {result['subject']}")
    print(f"\nQuestions-only PDF: {result['questions_only_pdf']}")
    print(f"Full PDF with answers: {result['full_pdf']}")
    print(f"\nGenerated {len(result['questions'])} questions total")

    # Display first few questions as preview
    for idx, q in enumerate(result['questions'][:3], start=1):
        print(f"\nPreview Question {idx}:")
        lines = q.split('\n')
        for line in lines:
            if not line.startswith('Correct Answer:') and not line.startswith('Explanation:'):
                print(line)


Course: ECC427 - Business for Engineers
Instructor: Dr. John Bush Idoko
Subject: Industrial Engineering

Questions-only PDF: ./ECC427_exam_questions.pdf
Full PDF with answers: ./ECC427_exam_with_answers.pdf

Generated 20 questions total

Preview Question 1:
Q:What is the primary focus of a Plant Engineer's responsibilities in a production activity?
A) Designing and implementing changes to the product
B) Running the support services of the plant and its activities
C) Conducting maintenance of the plant equipment
D) Managing the overall production process and strategy


Preview Question 2:
Q:What is the key distinction between the roles of a Design Engineer and a Maintenance Engineer in a production plant?
A) The Design Engineer focuses on product design, while the Maintenance Engineer focuses on plant equipment maintenance
B) The Design Engineer is responsible for maintenance, while the Maintenance Engineer designs new equipment
C) The Design Engineer specifies requirements for changes